In [21]:
import pandas as pd
import numpy as np

In [22]:
def filter_stocks(kse100_df, stock_data):
    """
    Filters stocks based on the provided criteria.
    
    Parameters:
        kse100_df (pd.DataFrame): DataFrame of KSE-100 stocks.
        stock_data (pd.DataFrame): DataFrame containing stock details (Free Float Market Cap, daily turnover, etc.).
    Returns:
        pd.DataFrame: Filtered DataFrame of top 15 stocks based on Free Float Market Cap.
    """
    # Step 1: Filter stocks part of the KSE-100 Index
    kse100_stocks = stock_data[stock_data['Stock'].isin(kse100_df['Stock'])]

    # Step 2: Filter top 30 companies based on Free Float Market Capitalization
    top30_stocks = kse100_stocks.nlargest(30, 'FreeFloatMarketCap')

    # Step 3: Filter stocks with average daily turnover in the last year >= 100,000 shares
    # top30_stocks['AverageDailyTurnover'] = top30_stocks['DailyTurnover'].apply(lambda x: sum(x) / len(x))
    top30_stocks['AverageDailyTurnover'] = top30_stocks['DailyTurnover']
    filtered_by_turnover = top30_stocks[top30_stocks['AverageDailyTurnover'] >= 100_000]

    # Step 4: Filter stocks not traded below 10,000 shares on any single day
    def meets_minimum_trading_condition(daily_turnover, listing_days):
        # return all(turnover >= 10_000 for turnover in daily_turnover[:min(len(daily_turnover), listing_days)])
        return daily_turnover>=10000

    filtered_by_minimum_trading = filtered_by_turnover[
        filtered_by_turnover.apply(
            lambda row: meets_minimum_trading_condition(row['DailyTurnover'], row['ListingDays']), axis=1
        )
    ]

    # Step 5: Select the top 15 stocks based on Free Float Market Capitalization
    top15_stocks = filtered_by_minimum_trading.nlargest(15, 'FreeFloatMarketCap')

    return top15_stocks

# Adjusted DataFrame from the CSV file
kse100_df = pd.read_csv('daily_kse100_data.csv')
kse100_df = kse100_df.rename(columns={
    'Symbol': 'Stock',
    'Freefloat (M)': 'FreeFloatMarketCap',
})
# Convert columns to numeric after replacing commas
kse100_df['FreeFloatMarketCap'] = pd.to_numeric(
    kse100_df['FreeFloatMarketCap'].astype(str).str.replace(',', ''), errors='coerce'
)
kse100_df['Volume'] = pd.to_numeric(
    kse100_df['Volume'].astype(str).str.replace(',', ''), errors='coerce'
)
kse100_df['LDCP'] = pd.to_numeric(kse100_df['LDCP'], errors='coerce')
# Calculate derived columns
kse100_df['DailyTurnover'] = kse100_df['Volume'] * kse100_df['LDCP']
# Add placeholder value
kse100_df['ListingDays'] = 365  # Placeholder value
# # Optional: Check for NaN values
# if kse100_df.isna().any().any():
#     print("Warning: The DataFrame contains NaN values.")

# NBPPGI as 'stock_data' DataFrame for simulation
stock_data = pd.read_csv('daily_nbppgi_data.csv')
stock_data = stock_data.rename(columns={
    'SYMBOL': 'Stock',
    'FF BASED MCAP': 'FreeFloatMarketCap',
})
# Convert columns to numeric after replacing commas
# stock_data['FreeFloatMarketCap'] = pd.to_numeric(
#     stock_data['FreeFloatMarketCap'].astype(str).str.replace(',', ''), errors='coerce'
# )
stock_data['Volume'] = stock_data['VOLUME']
stock_data['LDCP'] = pd.to_numeric(stock_data['PRICE'], errors='coerce')
# Calculate derived columns
stock_data['DailyTurnover'] = stock_data['Volume'] * stock_data['PRICE']
# Add placeholder value
stock_data['ListingDays'] = 365  # Placeholder value

# Apply the filter_stocks function
filtered_stocks = filter_stocks(kse100_df, stock_data)

# Display the results
print(filtered_stocks[['Stock', 'FreeFloatMarketCap', 'AverageDailyTurnover']])

    Stock  FreeFloatMarketCap  AverageDailyTurnover
3     FFC        2.939640e+11          1.606115e+09
14    UBL        1.671691e+11          7.326455e+08
10   OGDC        1.505045e+11          2.960117e+09
12    PPL        1.412505e+11          2.967441e+09
5    HUBC        1.253343e+11          2.804946e+09
1   EFERT        1.228869e+11          6.985493e+08
2   ENGRO        1.142692e+11          4.238907e+08
6    LUCK        1.088940e+11          6.932313e+08
13    SYS        1.088074e+11          2.368435e+08
7     MCB        1.073386e+11          1.373168e+08
8    MEBL        1.007164e+11          4.778793e+08
0    BAHL        9.530473e+10          3.370526e+07
4     HBL        9.366147e+10          6.560169e+08
11    POL        8.351219e+10          5.289303e+08
9     MTL        5.416766e+10          2.200278e+08


In [23]:
# def filter_stocks(kse100_df, stock_data):
#     """
#     Filters stocks based on the provided criteria.
    
#     Parameters:
#         kse100_df (pd.DataFrame): DataFrame of KSE-100 stocks.
#         stock_data (pd.DataFrame): DataFrame containing stock details (Free Float Market Cap, daily turnover, etc.).
        
#     Returns:
#         pd.DataFrame: Filtered DataFrame of top 15 stocks based on Free Float Market Cap.
#     """
#     # Step 1: Filter stocks part of the KSE-100 Index
#     kse100_stocks = stock_data[stock_data['Stock'].isin(kse100_df['Stock'])]

#     # Step 2: Filter top 30 companies based on Free Float Market Capitalization
#     top30_stocks = kse100_stocks.nlargest(30, 'FreeFloatMarketCap')

#     # Step 3: Filter stocks with average daily turnover in the last year >= 100,000 shares
#     top30_stocks['AverageDailyTurnover'] = top30_stocks['DailyTurnover'].apply(lambda x: sum(x) / len(x))
#     filtered_by_turnover = top30_stocks[top30_stocks['AverageDailyTurnover'] >= 100_000]

#     # Step 4: Filter stocks not traded below 10,000 shares on any single day
#     def meets_minimum_trading_condition(daily_turnover, listing_days):
#         return all(turnover >= 10_000 for turnover in daily_turnover[:min(len(daily_turnover), listing_days)])

#     filtered_by_minimum_trading = filtered_by_turnover[
#         filtered_by_turnover.apply(
#             lambda row: meets_minimum_trading_condition(row['DailyTurnover'], row['ListingDays']), axis=1
#         )
#     ]

#     # Step 5: Select the top 15 stocks based on Free Float Market Capitalization
#     top15_stocks = filtered_by_minimum_trading.nlargest(15, 'FreeFloatMarketCap')

#     return top15_stocks

# # Example usage
# # Load DF
# kse100_df = pd.read_csv('daily_kse100_data.csv')

# # kse100_df contains the list of stocks in the KSE-100 Index
# kse100_df = pd.DataFrame({'Stock': ['StockA', 'StockB', 'StockC', '...']})

# # Renaming relevant columns to match the function's expected format
# kse100_df = kse100_df.rename(columns={
#     'Symbol': 'Stock',
#     'Freefloat (M)': 'FreeFloatMarketCap',
#     'Volume': 'DailyTurnover'
# })

# # Converting relevant columns to numeric types for proper calculations
# kse100_df['FreeFloatMarketCap'] = pd.to_numeric(kse100_df['FreeFloatMarketCap'].str.replace(',', ''), errors='coerce')
# kse100_df['DailyTurnover'] = pd.to_numeric(kse100_df['DailyTurnover'].str.replace(',', ''), errors='coerce')

# # stock_data contains information for each stock
# # Each stock has a list of daily turnovers for the last year and the number of listing days
# stock_data = pd.DataFrame({
#     'Stock': ['StockA', 'StockB', 'StockC', 'StockD', '...'],
#     'FreeFloatMarketCap': [5000000, 3000000, 1000000, 2000000, '...'],
#     'DailyTurnover': [
#         [120000, 130000, 125000, 140000, '...'],  # Example turnovers for StockA
#         [90000, 110000, 100000, 95000, '...'],   # Example turnovers for StockB
#         [150000, 155000, 160000, 150000, '...'], # Example turnovers for StockC
#         [80000, 85000, 95000, 90000, '...'],     # Example turnovers for StockD
#     ],
#     'ListingDays': [365, 300, 365, 100, '...'],  # Example listing days for each stock
# })

# # Apply the filters
# filtered_stocks = filter_stocks(kse100_df, stock_data)

# # Display the result
# print(filtered_stocks)